In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [8]:
x = np.load("../data/data_40_500.npy")
print(x.shape)
x = x.reshape(-1, 40, 500, 1)
print(x.shape)
y = np.load("../data/norm_pad_15.npy")  # normailized
y = y.reshape(-1, 30)
y.shape

(13100, 40, 500)
(13100, 40, 500, 1)


(13100, 30)

In [9]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# Reshape to add the channel dimension
X_train_reshaped = X_train.reshape(X_train.shape[0], 40, 500, 1)
X_val_reshaped = X_val.reshape(X_val.shape[0], 40, 500, 1)


# # # Define a learning rate scheduler function
# def lr_scheduler(epoch, lr):
#     return lr * 0.99  # Adjust the multiplier as needed


# # Define early stopping callback
# early_stopping_callback = EarlyStopping(
#     patience=10,  # Number of epochs with no improvement after which training will be stopped
#     restore_best_weights=True,
#     monitor="val_loss",  # Metric to monitor (could be 'val_mse' or 'val_mae' based on your choice)
# )


# Build a simple CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation=None, input_shape=(40, 500, 1)))
model.add(Conv2D(16, kernel_size=(3, 3), activation=None, input_shape=(40, 500, 32)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation=None))
model.add(Dense(1))
model.add(Dense(128, activation=None))
model.add(Dense(30, activation=None))

# # # Use the Adam optimizer with the learning rate scheduler
# optimizer = Adam(learning_rate=0.002)  # Set the initial learning rate
# model.compile(optimizer=optimizer, loss="mse", metrics=["mse"])

# Use MAE as the loss function
optimizer = Adam(learning_rate=0.0025)
model.compile(optimizer=optimizer, loss="mae")

# Define the learning rate scheduler callback
# lr_scheduler_callback = LearningRateScheduler(lr_scheduler, verbose=1)

In [10]:
history = model.fit(
    X_train_reshaped,
    y_train,
    epochs=3,
    batch_size=64,
    validation_data=(X_val_reshaped, y_val),
    # callbacks=[lr_scheduler_callback],
)

Epoch 1/3


2023-12-04 14:26:07.456770: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.24GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-04 14:26:08.022205: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.24GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


164/164 [==============================] - ETA: 0s - loss: 11.6797

2023-12-04 14:26:25.789089: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-04 14:26:26.342139: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


164/164 [==============================] - 20s 95ms/step - loss: 11.6797 - val_loss: 0.1054
Epoch 2/3
164/164 [==============================] - 12s 72ms/step - loss: 0.1096 - val_loss: 0.1099
Epoch 3/3
164/164 [==============================] - 12s 73ms/step - loss: 0.1054 - val_loss: 0.1076


In [11]:
# Make predictions
predictions = model.predict(X_val_reshaped)

# Calculate MSE on validation set
mse_val = mean_squared_error(y_val, predictions)
print("Mean Squared Error on Validation Set:", mse_val)
predictions[100], y[100]
model.summary()

82/82 [==============================] - 2s 18ms/step
Mean Squared Error on Validation Set: 0.053969684927310586
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 38, 498, 32)       320       
                                                                 
 conv2d_2 (Conv2D)           (None, 36, 496, 16)       4624      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 18, 248, 16)       0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 71424)             0         
                                                                 
 dense_4 (Dense)             (None, 128)               9142400   
                                                                 
 dense_

In [12]:
import matplotlib.pyplot as plt

plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'